# Example of powering an interactive dashboard via Redshift

In [1]:
# Imports
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

import calendar
import pandas as pd
import numpy as np

In [2]:
# Build AppViewer 
from jupyterlab_dash import AppViewer
viewer = AppViewer()

In [3]:
# Local test - set up connection
%run ./db_common_local.py

conn = default_conn

In [ ]:
# Sagemaker notebook env
%run ./db_common.py

conn = default_conn

In [4]:
# Initialize
query_stmt1 = '''
    select  ca_state,
       count(distinct ws_order_number) as "order_count"
      ,avg(ws_ext_ship_cost) as "avg_shipping_cost"
      ,avg(ws_net_profit) as "avg_net_profit"
    from
       web_sales cs1
      ,date_dim
      ,customer_address
    where
        d_year = %s
    and cs1.ws_ship_date_sk = d_date_sk
    and cs1.ws_ship_addr_sk = ca_address_sk
    and cs1.ws_order_number not in (select wr_order_number
               from web_returns cr1)
    group by 1
    order by 2 desc;
'''

dataset = 'tpcds_100gb'

search_path = 'set search_path to %s;' % dataset
exec_dml(conn, search_path)

available_years = np.array([1998,1999,2000,2001,2002])


Escaped time: 0 seconds


In [5]:

app = dash.Dash("Redshift dashboard")
app.layout = html.Div(id='my-div', children=[
        html.Div([
            html.Div(children='Year: '),
            dcc.Dropdown(
                    id='input_year',
                    options=[{'label': i, 'value': i} for i in available_years],
                    value='1998'
                )
        ],
        style={'width': '90%', 'display': 'inline-block'}),
        dcc.Graph(
            id='my-graph-id',
            figure=dict(
                data=[],
                layout={},
            ),
        )
    ])

@app.callback(
    dash.dependencies.Output('my-graph-id', 'figure'),
    [dash.dependencies.Input('input_year', 'value')]
)

def update_graph(input_year):
    query = query_stmt1 % input_year
    query = query.replace('\n', ' ').replace('\r', '')
    df = exec_query(conn, query)
    # convert everything to string for display purpose
    for col in df.columns: 
        df[col] = df[col].astype(str)

    df['text'] = 'order ' + df['order_count'] + '<br>' + \
                'average shipping cost ' + df['avg_shipping_cost'] + '<br>' + \
                'average net profit ' + df['avg_net_profit']

    fig = go.Figure(data=go.Choropleth(
        locations=df['ca_state'], # Spatial coordinates
        z = df['order_count'].astype(float), # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        text = df['text'],
        colorscale = 'Reds',
        colorbar_title = "Total orders",
    ))

    fig.update_layout(
        title_text = 'US Shipping cost and profit by State',
        geo_scope='usa', # limite map scope to USA
    )
    return fig
    
viewer.show(app)